In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd

# Weather API Call

In [1]:
import requests

###Getting weather data for melbourne aiport from the open weather api for current weather

lat= -37.663712

lon= 144.844788

api_key='9efa0a3dd9883bae2b9f3bda6eff2e36'



#example url to call: https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={API key}



url = "https://api.openweathermap.org/data/2.5/weather?"



params = {'lat': str(lat), 'lon':str(lon), "appid":api_key}



response = requests.get(url, params=params)



json_response = response.json()



json_response

{'coord': {'lon': 144.8448, 'lat': -37.6637},
 'weather': [{'id': 801,
   'main': 'Clouds',
   'description': 'few clouds',
   'icon': '02n'}],
 'base': 'stations',
 'main': {'temp': 285.88,
  'feels_like': 284.85,
  'temp_min': 284.29,
  'temp_max': 286.61,
  'pressure': 1020,
  'humidity': 63,
  'sea_level': 1020,
  'grnd_level': 1005},
 'visibility': 10000,
 'wind': {'speed': 4.63, 'deg': 360},
 'clouds': {'all': 24},
 'dt': 1724320692,
 'sys': {'type': 1,
  'id': 9554,
  'country': 'AU',
  'sunrise': 1724273785,
  'sunset': 1724313063},
 'timezone': 36000,
 'id': 2164495,
 'name': 'Greenvale',
 'cod': 200}

In [2]:
import pandas as pd

file_path = '~/code/diego-mogollon/lateguru/lateguru/raw_data/All_Airports.csv'

preprocessed_df = pd.read_csv(file_path, compression='zip')

preprocessed_df.head()

,Time,Origin,Dest,Carrier,Cancelled,CancellationReason,Delayed,DepDelayMinutes,CarrierDelay,WeatherDelay,...,LateAircraftDelay,Temperature,Feels_Like_Temperature,Altimeter_Pressure,Sea_Level_Pressure,Visibility,Wind_Speed,Wind_Gust,Precipitation,Ice_Accretion_3hr
0,2021-01-01 09:00:00,LAX,JFK,American Airlines Inc.,False,NaN,False,0.0,NaN,NaN,...,NaN,56.0,55.9,1020.32,1020.1,16093.40,0.0,24.97,0.0,0.0
1,2021-01-02 09:00:00,LAX,JFK,American Airlines Inc.,False,NaN,True,2.0,NaN,NaN,...,NaN,54.0,54.0,1025.74,1025.5,16093.40,0.0,24.97,0.0,0.0
2,2021-01-03 09:00:00,LAX,JFK,American Airlines Inc.,False,NaN,True,28.0,28.0,0.0,...,0.0,52.0,52.0,1024.38,1015.2,4828.02,4.6,24.97,0.0,0.0
3,2021-01-03 09:00:00,LAX,JFK,American Airlines Inc.,False,NaN,True,28.0,28.0,0.0,...,0.0,53.0,53.1,1024.38,1024.1,6437.36,4.6,24.97,0.0,0.0
4,2021-01-04 09:00:00,LAX,JFK,American Airlines Inc.,False,NaN,False,0.0,NaN,NaN,...,NaN,51.0,51.1,1023.03,1015.2,6437.36,4.6,24.97,0.0,0.0


In [3]:
preprocessed_df.describe(include='all')

,Time,Origin,Dest,Carrier,Cancelled,CancellationReason,Delayed,DepDelayMinutes,CarrierDelay,WeatherDelay,...,LateAircraftDelay,Temperature,Feels_Like_Temperature,Altimeter_Pressure,Sea_Level_Pressure,Visibility,Wind_Speed,Wind_Gust,Precipitation,Ice_Accretion_3hr
count,15144514,15144514,15144514,15144514,15144514,339711,15144514,1.481562e+07,3.274328e+06,3.274328e+06,...,3.274328e+06,1.514451e+07,1.514451e+07,1.514451e+07,1.514451e+07,1.514451e+07,1.514451e+07,1.514451e+07,1.514451e+07,1.514451e+07
unique,24032,30,352,17,2,4,2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,2022-11-04 13:00:00,LAX,ATL,Delta Air Lines Inc.,False,Weather,False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1713,1300942,503262,2490622,14804803,195469,9249063,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.608864e+01,2.551989e+01,5.424442e+00,...,2.440234e+01,6.317238e+01,6.218925e+01,1.016327e+03,1.016158e+03,1.452299e+04,9.068776e+00,2.608714e+01,1.739791e-01,1.685416e-04
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.152882e+01,6.765369e+01,3.479500e+01,...,5.611603e+01,1.805368e+01,2.100936e+01,6.502375e+00,6.172797e+00,3.031896e+04,5.421863e+00,3.098764e+00,1.295235e+00,2.063771e-02
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,-2.200000e+01,-4.086000e+01,1.164900e+02,9.078000e+02,0.000000e+00,0.000000e+00,9.210000e+00,0.000000e+00,0.000000e+00
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,5.100000e+01,5.110000e+01,1.012530e+03,1.012800e+03,1.609340e+04,5.750000e+00,2.474000e+01,0.000000e+00,0.000000e+00
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,6.000000e+00,0.000000e+00,...,0.000000e+00,6.500000e+01,6.490000e+01,1.016260e+03,1.016200e+03,1.609340e+04,8.060000e+00,2.624000e+01,0.000000e+00,0.000000e+00
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.100000e+01,2.600000e+01,0.000000e+00,...,2.700000e+01,7.600000e+01,7.600000e+01,1.020320e+03,1.019300e+03,1.609340e+04,1.266000e+01,2.762000e+01,0.000000e+00,0.000000e+00


In [7]:
preprocessed_df['CancellationReason'].unique()

array([nan, 'Weather', 'Carrier', 'Security', 'National Air System'],
      dtype=object)

In [4]:
preprocessed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15144514 entries, 0 to 15144513
Data columns (total 22 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   Time                    object 
 1   Origin                  object 
 2   Dest                    object 
 3   Carrier                 object 
 4   Cancelled               bool   
 5   CancellationReason      object 
 6   Delayed                 bool   
 7   DepDelayMinutes         float64
 8   CarrierDelay            float64
 9   WeatherDelay            float64
 10  NASDelay                float64
 11  SecurityDelay           float64
 12  LateAircraftDelay       float64
 13  Temperature             float64
 14  Feels_Like_Temperature  float64
 15  Altimeter_Pressure      float64
 16  Sea_Level_Pressure      float64
 17  Visibility              float64
 18  Wind_Speed              float64
 19  Wind_Gust               float64
 20  Precipitation           float64
 21  Ice_Accretion_3hr       float

# Building the FAST API